In [416]:
import os
import pandas as pd
import math
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', -1)

In [417]:
df1 = pd.read_csv("Avg Temp Parsing.csv")
df2 = pd.read_csv("Crime Data Parsing.csv")
df3 = pd.read_csv("HighGrad Parsing.csv")
df4 = pd.read_csv("Political Parsed.csv")
df5 = pd.read_csv("Housing Parsing2.csv")
#df1 = df1.sort_values('Facility ID')
#df2 = df2.sort_values('Facility ID')
#df1 = df1[~df1['Measure Name'].str.contains('Device Days')]
#df1 = df1[~df1['Measure Name'].str.contains('Confidence Limit')]
#df1 = df1[~df1['Measure Name'].str.contains('Number of')]
#df1 = df1[~df1['Measure Name'].str.contains('Predicted Cases')]
#df1 = df1[~df1['Measure Name'].str.contains('Observed Cases')]
#df1 = df1[~df1['Measure Name'].str.contains('Patient Days')]

us_state_abbrev = {
    'Alabama': 'AL',
    'Alaska': 'AK',
    'Arizona': 'AZ',
    'Arkansas': 'AR',
    'California': 'CA',
    'Colorado': 'CO',
    'Connecticut': 'CT',
    'Delaware': 'DE',
    'District of Columbia': 'DC',
    'Florida': 'FL',
    'Georgia': 'GA',
    'Hawaii': 'HI',
    'Idaho': 'ID',
    'Illinois': 'IL',
    'Indiana': 'IN',
    'Iowa': 'IA',
    'Kansas': 'KS',
    'Kentucky': 'KY',
    'Louisiana': 'LA',
    'Maine': 'ME',
    'Maryland': 'MD',
    'Massachusetts': 'MA',
    'Michigan': 'MI',
    'Minnesota': 'MN',
    'Mississippi': 'MS',
    'Missouri': 'MO',
    'Montana': 'MT',
    'Nebraska': 'NE',
    'Nevada': 'NV',
    'New Hampshire': 'NH',
    'New Jersey': 'NJ',
    'New Mexico': 'NM',
    'New York': 'NY',
    'North Carolina': 'NC',
    'North Dakota': 'ND',
    'Northern Mariana Islands':'MP',
    'Ohio': 'OH',
    'Oklahoma': 'OK',
    'Oregon': 'OR',
    'Palau': 'PW',
    'Pennsylvania': 'PA',
    'Puerto Rico': 'PR',
    'Rhode Island': 'RI',
    'South Carolina': 'SC',
    'South Dakota': 'SD',
    'Tennessee': 'TN',
    'Texas': 'TX',
    'Utah': 'UT',
    'Vermont': 'VT',
    'Virgin Islands': 'VI',
    'Virginia': 'VA',
    'Washington': 'WA',
    'West Virginia': 'WV',
    'Wisconsin': 'WI',
    'Wyoming': 'WY',
}

pop = pd.read_csv("Population.csv", encoding = "ISO-8859-1")
pop["State"] = pop["State"].map(us_state_abbrev)
pop["County"] = pop["County"].str.replace("County", "")
pop["County"] = pop["County"].str.replace("Parish", "")
pop["County"] = pop["County"].str.replace("Borough", "")
pop["County"] = pop["County"].str.strip()
pop["County"] = pop["County"].str.title()
pop["County, State"] = pop["County"] + ", " + pop["State"]
pop = pop.drop(["County", "State"], axis=1)

In [418]:
df1["Type"] = "Temp"
df2["Type"] = "Crime" 
df3["Type"] = "GradRate"
df4["Type"] = "Political"
df5["Type"] = "HousPrices"

In [419]:
temp = [df1,df2,df3,df4,df5]
findata = pd.concat(temp, sort = True)
findata.head()

,"County, State",Type,Value
0,"Autauga, AL",Temp,66.0
1,"Baldwin, AL",Temp,68.7
2,"Barbour, AL",Temp,66.8
3,"Bibb, AL",Temp,64.4
4,"Blount, AL",Temp,63.2


In [420]:
findata.to_csv("findata.csv", index = False)

In [421]:
merge = pd.merge(df3, pd.merge(df1, df2, on="County, State"), on="County, State")
merge = merge.rename(columns={"Value": "GradRate", "Value_x": "Temp", "Value_y": "Crime"})
merge = merge.drop(["Type", "Type_x", "Type_y"], axis=1)
merge = pd.merge(pd.merge(merge, df4, on="County, State"), df5, on="County, State")
merge = merge.rename(columns={"Value_x": "Political", "Value_y": "HousPrices"})
merge = merge.drop(["Type_x", "Type_y"], axis=1)
#merge = pd.merge(merge, pop, on="County, State")
merge = merge.dropna().reset_index(drop=True)

In [422]:
counties = merge["County, State"]
merge = merge.drop("County, State", axis=1)
merge = (merge-merge.min())/(merge.max()-merge.min())

In [448]:
clusters = 180

from sklearn.cluster import KMeans, AffinityPropagation
#affinity = AffinityPropagation().fit(merge)
kmeans = KMeans(n_clusters=clusters, random_state=0).fit(merge)

In [487]:
labels = kmeans.labels_

clustering = {}
array = []

for i in range(clusters):
    array.append([])
    
for i in range(len(labels)):
    clustering[str(i)] = dict(merge.iloc[i])
    clustering[str(i)]["label"] = labels[i]

clusteredData = pd.DataFrame(clustering).transpose()
clusteredData = clusteredData.set_index(counties.index)
clusteredData["County, State"] = counties

def getRelatedCounties(county):
    return clusteredData[clusteredData["label"] == clusteredData[clusteredData["County, State"] == county].iloc[0]["label"]]
                         

In [504]:
county = "Cobb, GA"
getRelatedCounties(county)

,Crime,GradRate,HousPrices,Political,Temp,label,"County, State"
117,0.175498,0.765625,0.100846,0.517238,0.588764,34.0,"Butte, CA"
309,0.150563,0.750000,0.093794,0.480243,0.644944,34.0,"Cobb, GA"
321,0.157103,0.796875,0.057472,0.426906,0.658427,34.0,"Douglas, GA"
337,0.126061,0.734375,0.081453,0.457394,0.635955,34.0,"Gwinnett, GA"
362,0.206915,0.750000,0.053068,0.479228,0.674157,34.0,"Newton, GA"
1021,0.218642,0.765625,0.028914,0.494602,0.647191,34.0,"Panola, MS"
1300,0.088693,0.781250,0.141396,0.431777,0.606742,34.0,"Chatham, NC"
1318,0.199477,0.703125,0.077130,0.509431,0.584270,34.0,"Granville, NC"
1342,0.202385,0.734375,0.049542,0.478230,0.642697,34.0,"Pasquotank, NC"
1346,0.227587,0.765625,0.046157,0.446279,0.653933,34.0,"Pitt, NC"
